In [1]:
from random import uniform
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import random, pickle
from tqdm import tqdm

## Configs

In [2]:
# chormosome의 개수
POPULATION = 100

# 좋은 chormosome의 개수
GOOD_PARENTS_CNT = 30

# 운좋은 chormosome의 개수
LUCKY_PARENTS_CNT = 10

# 각 parent가 만드는 chromosome 개수
CHILDREN_CNT = 5

# 돌연변이 생성 비율
MUTATION_PROB = 0.1

# 최대 세대 수 
MAX_GENERATIONS = 50

## Load the image and flatten it

In [3]:
# Load Image
jpg_img_arr = mpimg.imread('Bigger-Splash-1967.jpg')
jpg_IMG = Image.open('Bigger-Splash-1967.jpg')
FLATTENED = jpg_img_arr.flatten()
LEN_FLATTENED = len(FLATTENED)
# print("flattend size :", FLATTENED.shape)

def save_pickle(data, filename):
    with open("{}.pickle".format(filename), 'wb') as t:
        pickle.dump(data, t)
    print("PICKLE SAVE DONE")

## Initial Population

In [4]:
class chromoSome:
    def __init__(self, chromo_data=None):
        if chromo_data is None:
            self.chromo = [np.random.randint(255) for _ in range(len(FLATTENED))]
        else:
            self.chromo = chromo_data
    
    def __repr__(self):
        return "chromosome fitenss : {}".format(self.fitness)
    
    @property
    def fitness(self):
        # score가 0에 가까우면 좋은 것
        score = 0
        for gene, wanted in zip(self.chromo, FLATTENED):
            score += abs(gene - wanted)
        
        return score

class Generation:
    cnt = 0
    def __init__(self, population):
        Generation.cnt += 1
        self.generation_lv = Generation.cnt
        self.population = population
        self.sorted_pop = self.sort_pop()
    
    def __repr__(self):
        return "Generation Level : {}".format(self.generation_lv)
    
 
    def evolution(self):
        print("Start Evolution Generation level %d" % Generation.cnt)
        # next_generation = list()
        # select Good_parents
        good_parents = self.sorted_pop[:GOOD_PARENTS_CNT]
        
        # select Lucky_parents
        lucky_parents = random.sample(self.sorted_pop[GOOD_PARENTS_CNT:], LUCKY_PARENTS_CNT)
        
#         print("good :", good_parents)
#         print("lucky :", lucky_parents)
        
        # make Childeren
        good_child = self.make_children(good_parents)
        lucky_child = self.make_children(lucky_parents)
        
        # merge Childeren
        children = good_child + lucky_child
        random.shuffle(children)
        
        # make mutations
        children = self.make_mutation(children)
                
        return Generation(children)
    
    def sort_pop(self):
        sorted_pop = sorted(self.population, key=lambda x: x.fitness, reverse=False)
        
        return sorted_pop
    
    def make_children(self, parents):
        children = list()
        
        # crossover
        for i in range(int(len(parents) / 2)):
            # 한개는 중간에서 반반 섞기
            split_child = chromoSome(parents[i].chromo[:int(LEN_FLATTENED/2)] + parents[len(parents)-1-i].chromo[:int(LEN_FLATTENED/2)])
            children.append(split_child)
            
            # 나머지는 랜덤으로 가져오기
            for _ in range(CHILDREN_CNT - 1):
                random_child = chromoSome(random.sample(parents[i].chromo + parents[len(parents)-1-i].chromo, LEN_FLATTENED))
                children.append(random_child)
        
        return children
    
    @property
    def mean_fitness(self):
        return np.mean([chromosome.fitness for chromosome in self.population])
    
    @property
    def get_best(self):
        return sorted(self.population, key=lambda x: x.fitness, reverse=True)[0]
    
    
    def make_mutation(self, children):
        # mutated = list()
        mutation_cnt = int(len(children) * MUTATION_PROB)
        for i in range(mutation_cnt):
            children[i] = chromoSome([np.random.randint(255) for _ in range(LEN_FLATTENED)])
        
        return children
    
    def fitness(self):
        return np.mean([chromo.fitness for chromo in self.population])

In [ ]:
# first generation

i = 0
initial_population = [chromoSome() for _ in range(POPULATION)]
gen = Generation(initial_population)
print(gen)


mean_fitness_list = list()
for num in tqdm(range(MAX_GENERATIONS)):
    gen = gen.evolution()
    mean_fitness = gen.mean_fitness
    print("mean fitness : {}".format(mean_fitness))
    mean_fitness_list.append(mean_fitness)

best_chromosome = gen.get_best

save_pickle(mean_fitness_list, "mean_fitness_list")
save_pickle(best_chromosome, "best_chromosome")

Generation Level : 1


  0%|          | 0/50 [00:00<?, ?it/s]

Start Evolution Generation level 1


In [ ]:
shapen = np.reshape(best_chromosome.chromo,jpg_img_arr.shape)
img = Image.fromarray(shapen, 'RGB')
img